In [1]:
!pip install torch

In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    # Training settings
    batch_size=64
    test_batch_size=1000
    epochs=14
    lr=1.0
    gamma=0.7
    no_cuda=False
    seed=1
    log_interval=10

    save_model=False
    use_cuda = not no_cuda and torch.cuda.is_available()

    torch.manual_seed(seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train(log_interval, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")

main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.327989
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.396983
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.689284
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.539597
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.313588
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.390208
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.351364
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.236261
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.390681
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.358736
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.444956
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.207953
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.177445
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.267496
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.255226
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.136705
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.080729
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.223787
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.522845
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.234125
Train Epoch: 1 [12800/60000 (